In [ ]:
#data collection
import os
import cv2

# Define a new folder for A-Z data
DATA_DIR = 'D:/ML_go/Data_AZ'
os.makedirs(DATA_DIR, exist_ok=True)  # Create the folder if it doesn't exist

# Collect data for A-Z (26 letters)
classes = [chr(i) for i in range(ord('A'), ord('Z')+1)]
dataset_size = 100  # Number of images per letter

cap = cv2.VideoCapture(0)

for letter in classes:
    letter_dir = os.path.join(DATA_DIR, letter)
    os.makedirs(letter_dir, exist_ok=True)  # Create folder for each letter

    print(f'\n📷 Collecting data for letter: {letter}')
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Show the current letter in the center of the frame
        cv2.putText(frame, f'Letter: {letter}', (100, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3, cv2.LINE_AA)
        cv2.putText(frame, 'Press "Q" to start!', (100, 200), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('Sign Language Data Collection', frame)

        if cv2.waitKey(1) == ord('q'):
            break  # Start capturing images when 'Q' is pressed

    counter = 0
    while counter < dataset_size:
        ret, frame = cap.read()
        if not ret:
            break

        # Show progress on screen
        cv2.putText(frame, f'Capturing {letter}: {counter+1}/{dataset_size}', (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

        cv2.imshow('Sign Language Data Collection', frame)
        cv2.imwrite(os.path.join(letter_dir, f'{counter}.jpg'), frame)

        counter += 1
        cv2.waitKey(1)

cap.release()
cv2.destroyAllWindows()


In [8]:
#alphabet preprocessing
import os
import cv2
import pickle
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

# Path to dataset
DATA_DIR = 'D:/ML_go/Data_AZ'

# Labels (A-Z)
classes = [chr(i) for i in range(ord('A'), ord('Z')+1)]

# Lists to store data
data = []
labels = []

for letter in classes:
    letter_dir = os.path.join(DATA_DIR, letter)

    for image_file in os.listdir(letter_dir):
        image_path = os.path.join(letter_dir, image_file)

        # Read and convert image
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Process with MediaPipe
        results = hands.process(img_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                x_ = [lm.x for lm in hand_landmarks.landmark]
                y_ = [lm.y for lm in hand_landmarks.landmark]

                # Normalize landmarks
                min_x, min_y = min(x_), min(y_)
                landmarks = [(x - min_x, y - min_y) for x, y in zip(x_, y_)]
                
                # Flatten into a feature vector
                feature_vector = np.array(landmarks).flatten()
                
                # Store data
                data.append(feature_vector)
                labels.append(letter)

# Save processed data
with open("letters1.pkl", "wb") as f:
    pickle.dump({"data": data, "labels": labels}, f)

print("✅ Hand landmarks extracted and saved as letters1.pkl")


✅ Hand landmarks extracted and saved as letters1.pkl


In [10]:
#alphabet trainig
import pickle
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load preprocessed data
with open("letters1.pkl", "rb") as f:
    dataset = pickle.load(f)

data = np.array(dataset["data"])
labels = np.array(dataset["labels"])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"✅ Model trained with accuracy: {accuracy:.2f}")

# Save the trained model
with open("letter_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("🎯 Model saved as letter_model.pkl")


✅ Model trained with accuracy: 1.00
🎯 Model saved as letter_model.pkl


In [11]:
#alphabet prediction
import cv2
import pickle
import numpy as np
import mediapipe as mp

# Load the trained model
with open("letter_model.pkl", "rb") as f:
    model = pickle.load(f)

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Open webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to RGB and process with MediaPipe
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    # Prepare data for prediction
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            x_ = [lm.x for lm in hand_landmarks.landmark]
            y_ = [lm.y for lm in hand_landmarks.landmark]

            # Normalize landmarks
            min_x, min_y = min(x_), min(y_)
            landmarks = [(x - min_x, y - min_y) for x, y in zip(x_, y_)]
            
            # Convert to a feature vector
            feature_vector = np.array(landmarks).flatten().reshape(1, -1)

            # Predict the letter
            prediction = model.predict(feature_vector)[0]

            # Display prediction
            cv2.putText(frame, f'Prediction: {prediction}', (50, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Show the frame
    cv2.imshow("Sign Language Prediction", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#word

In [ ]:
#word data collection
import cv2
import os

# Define dataset storage path
DATASET_PATH = "sign_data"  # Folder where all images will be stored

# List of 10 words
WORDS = ["Hello", "Yes", "No", "ThankYou", "Sorry", "Please", "Help", "Good", "Bad", "Stop"]

# Initialize camera
cap = cv2.VideoCapture(0)  # 0 for default webcam
print("Press 'S' to start collecting images.")
print("Press 'Q' to quit at any time.")

for sign_name in WORDS:
    sign_folder = os.path.join(DATASET_PATH, sign_name)

    # Create directory if it doesn't exist
    if not os.path.exists(sign_folder):
        os.makedirs(sign_folder)

    print(f"\nGet ready for '{sign_name}'. Press 'S' to start collecting images.")
    
    # Wait for 'S' to start collecting for the current word
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        cv2.putText(frame, f"Ready for {sign_name} - Press 'S' to start", (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.imshow("Data Collection", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('s'):
            break
        elif key == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            exit()

    count = 0
    print(f"Collecting images for '{sign_name}'... Press 'Q' to skip.")

    while count < 100:
        ret, frame = cap.read()
        if not ret:
            break

        # Display progress
        cv2.putText(frame, f"{sign_name} ({count}/100)", (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.imshow("Data Collection", frame)

        key = cv2.waitKey(1) & 0xFF

        # Capture images automatically
        img_path = os.path.join(sign_folder, f"{count}.jpg")
        cv2.imwrite(img_path, frame)
        count += 1

        # Skip to next word if 'Q' is pressed
        if key == ord('q'):
            print(f"Skipping '{sign_name}'...")
            break

    print(f"Completed collection for '{sign_name}'.")

cap.release()
cv2.destroyAllWindows()
print("Data collection finished for all words!")


In [12]:
#word preprocessing
import os
import cv2
import pickle
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

# Function to preprocess data
def preprocess_data(DATA_DIR, labels_list, output_file):
    data = []
    labels = []
    
    for label in labels_list:
        folder_path = os.path.join(DATA_DIR, label)
        
        for image_file in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_file)
            img = cv2.imread(image_path)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            results = hands.process(img_rgb)
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    x_ = [lm.x for lm in hand_landmarks.landmark]
                    y_ = [lm.y for lm in hand_landmarks.landmark]
                    
                    # Normalize landmarks
                    min_x, min_y = min(x_), min(y_)
                    landmarks = [(x - min_x, y - min_y) for x, y in zip(x_, y_)]
                    
                    # Flatten into a feature vector
                    feature_vector = np.array(landmarks).flatten()
                    
                    data.append(feature_vector)
                    labels.append(label)
    
    # Save preprocessed data
    with open(output_file, "wb") as f:
        pickle.dump({"data": data, "labels": labels}, f)
    print(f"✅ Data saved as {output_file}")

# Paths
WORD_DATA_DIR = "sign_data"  # Path where word images are stored
words = ["Hello", "Yes", "No", "ThankYou", "Sorry", "Please", "Help", "Good", "Bad", "Stop"]

# Preprocess and save
preprocess_data(WORD_DATA_DIR, words, "words_data.pkl")


✅ Data saved as words_data.pkl


In [13]:
#word training
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load preprocessed word data
with open("words_data.pkl", "rb") as f:
    data_dict = pickle.load(f)

X = np.array(data_dict["data"])
y = np.array(data_dict["labels"])

# Split data into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Test the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"✅ Model trained with accuracy: {accuracy:.2f}")

# Save the trained model
with open("sign_language_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("✅ Model saved as sign_language_model.pkl")


✅ Model trained with accuracy: 1.00
✅ Model saved as sign_language_model.pkl


In [14]:
#real time word prediction
import pickle
import cv2
import mediapipe as mp
import numpy as np

# Load trained model
with open("sign_language_model.pkl", "rb") as f:
    model = pickle.load(f)

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.5)

# Open webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip frame horizontally for a natural view
    frame = cv2.flip(frame, 1)
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process hand landmarks
    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            x_ = [lm.x for lm in hand_landmarks.landmark]
            y_ = [lm.y for lm in hand_landmarks.landmark]

            # Normalize coordinates
            min_x, min_y = min(x_), min(y_)
            landmarks = [(x - min_x, y - min_y) for x, y in zip(x_, y_)]

            # Convert to numpy array
            feature_vector = np.array(landmarks).flatten().reshape(1, -1)

            # Predict using trained model
            prediction = model.predict(feature_vector)[0]

            # Display prediction
            cv2.putText(frame, f'Prediction: {prediction}', (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show webcam feed
    cv2.imshow("Sign Language Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
